These are working functions, but are not optimized. Each function takes a pandas DataFrame as input with a true class label and predicted probabilities.

## Cumulative Captured Response

Returns both a dictionary and dataframe with underlying data for captured response. Can easily plot the cumulative caputured response curve using output.

In [ ]:
# define CCR function:
def calc_ccr(df, true_class):
    df = df.copy()
    
    '''
    Need to move the true class to the front of the dataframe
    '''
    temp_class = df[true_class]
    
    if 'true_class' in df:
        df.drop('true_class', axis = 1, inplace = True)
    else:
        df.drop(true_class, axis = 1, inplace = True)
        
    df.insert(0, 'true_class', temp_class)
    del temp_class
    
    ccr_dict = {}
    
    p_names = df.iloc[:, 1:].columns
    
    for x in p_names:
        df = df.sort_values(x, ascending = False)
        ccr_df = pd.DataFrame([i for i in range(1, df.shape[0] + 1, 1)])
        ccr_df[x] = np.array(df[x])
        ccr_df['true_class'] = np.array(df['true_class'])
        ccr_df.columns = ['n', 'prob', 'tc']
        ccr_df['cum_positives'] = ccr_df['tc'].cumsum()
        ccr_df['perc_samples_found'] = ccr_df['cum_positives'] / ccr_df['tc'].sum()
        ccr_df['perc_samples_tested'] = ccr_df['n'] / ccr_df['n'].max()
        ccr_df['model'] = x
        ccr_dict[x] = ccr_df
        
    keys = list(ccr_dict.keys())
    
    # initialize dataframe with first model
    ccr_frame = pd.DataFrame(ccr_dict[keys[0]])
    
    for key in keys[1:]:
        ccr_frame = pd.concat([ccr_frame, ccr_dict[key]])
    
    return list([ccr_dict, ccr_frame])

## Lift Curve

Takes a pandas DataFrame as input, with a true class and predicted probabilities. The 'cum' argument allows user to specify whether cumulative lift should be calculated.

In [2]:
def calc_lift(df, true_class, cum = False):
    df = df.copy()
    
    base_rate = df[true_class].sum() / df[true_class].count()
    
    '''
    Need to move the true class to the front of the dataframe
    '''
    
    temp_class = df[true_class]
    
    if 'true_class' in df:
        df.drop('true_class', axis = 1, inplace = True)
    else:
        df.drop(true_class, axis = 1, inplace = True)
        
    df.insert(0, 'true_class', temp_class)
    del temp_class
    
    def pct_rank_qcut(series, n):
        edges = pd.Series([float(i) / n for i in range(n + 1)])
        f = lambda x: (edges >= x).argmax()
        return series.rank(pct = True).apply(f)
    
    lift_dict = {}
    p_names = df.iloc[:, 1:].columns
    
    for x in p_names:
        df = df.sort_values(x, ascending = False)
        df['bin'] = pct_rank_qcut(df[x], 100)
        lift_df_1 = pd.DataFrame(df.groupby('bin')[x].max())
        lift_df_2 = pd.DataFrame(df.groupby('bin')[x].count())
        lift_df_3 = pd.DataFrame(df.groupby('bin')['true_class'].sum())
        
        lift_df = pd.merge(lift_df_1, lift_df_2, left_index = True, right_index = True)
        lift_df = pd.merge(lift_df, lift_df_3, left_index = True, right_index = True)
        lift_df.columns = ['prob_thres','n','positives']
        lift_df['random'] = base_rate * lift_df['n']
        lift_df['lift'] = ((lift_df['positives'] * 1.0) / lift_df['n']) / base_rate
        lift_df = lift_df.sort_values('prob_thres', ascending = False)
        lift_df['rand_perc'] = lift_df['random'].cumsum() / lift_df['random'].sum()
        if cum == True:
            lift_df['cum_n'] = lift_df['n'].cumsum()
            lift_df['cum_positives'] = lift_df['positives'].cumsum()
            lift_df['cum_random'] = base_rate * lift_df['cum_n']
            lift_df['cum_lift'] = ((lift_df['cum_positives'] * 1.0) / lift_df['cum_n']) / base_rate
            
        lift_df['model'] = x
        lift_dict[x] = lift_df
        
    keys = list(lift_dict.keys())
    
    # initialize dataframe with first model
    lift_frame = pd.DataFrame(lift_dict[keys[0]])
    
    for key in keys[1:]:
        lift_frame = pd.concat([lift_frame, lift_dict[key]])
        
    return list([lift_dict, lift_frame])

## Model Lift

Takes a pandas DataFrame as input with a true class and predicted probabilites. Calculates the lift for each model given a specified number of samples. 

In [3]:
def model_lift(df, true_class, n):
    
    df = df.copy()
    base_rate = df[true_class].sum() / df[true_class].count()
    
    '''
    Need to move the true class to the front of the dataframe
    '''
    
    temp_class = df[true_class]
    
    if 'true_class' in df:
        df.drop('true_class', axis = 1, inplace = True)
    else:
        df.drop(true_class, axis = 1, inplace = True)
        
    df.insert(0, 'true_class', temp_class)
    del temp_class
    
    lift_dict = {}
    
    for col in df.columns[1:]:
        true_pos = df.sort_values(by = col, ascending = False).iloc[0:n-1, 0].sum()
        expected_pos = base_rate * n
        lift_dict[col] = '%.2f'%(true_pos / expected_pos)
        
    return lift_dict